In [ ]:
# Enhanced FX Barrier Option Pricer for American Up-and-Out Options
import time
from typing import Any, Callable, Dict, List, Tuple
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

class OptimizedAmericanBarrierPricer:
    """
    Optimized Monte Carlo pricer for American up-and-out FX barrier options
    Focus on accuracy to the fourth decimal place
    """

    def __init__(
        self,
        num_paths: int = 500000,  # Increased for fourth decimal accuracy
        num_steps: int = 252,
        seed: int = 42,
        antithetic: bool = True,
        control_variate: bool = True,
    ):
        self.num_paths = num_paths
        self.num_steps = num_steps
        self.seed = seed
        self.antithetic = antithetic
        self.control_variate = control_variate
        np.random.seed(seed)

    def generate_paths(
        self,
        S0_K: float,
        T: float,
        r: float,
        sigma: float,
        sigma_curve: Callable[[np.ndarray], np.ndarray] = None,
    ) -> np.ndarray:
        """Generate optimized price paths with higher precision"""
        dt = T / self.num_steps
        effective_paths = self.num_paths // 2 if self.antithetic else self.num_paths

        # Use higher precision random numbers
        Z = np.random.normal(0, 1, size=(effective_paths, self.num_steps)).astype(np.float64)

        if self.antithetic:
            paths = np.zeros((self.num_paths, self.num_steps + 1), dtype=np.float64)
        else:
            paths = np.zeros((effective_paths, self.num_steps + 1), dtype=np.float64)

        paths[:, 0] = S0_K
        time_grid = np.linspace(0, T, self.num_steps + 1)

        for t in range(1, self.num_steps + 1):
            if sigma_curve is not None:
                vol = sigma_curve(time_grid[t])
            else:
                vol = sigma

            drift = (r - 0.5 * vol**2) * dt
            diffusion = vol * np.sqrt(dt)

            if self.antithetic:
                increment_1 = drift + diffusion * Z[:, t - 1]
                increment_2 = drift - diffusion * Z[:, t - 1]

                paths[:effective_paths, t] = paths[:effective_paths, t - 1] * np.exp(increment_1)
                paths[effective_paths:, t] = paths[effective_paths:, t - 1] * np.exp(increment_2)
            else:
                paths[:, t] = paths[:, t - 1] * np.exp(drift + diffusion * Z[:, t - 1])

        return paths

    def longstaff_schwartz_barrier(
        self,
        paths: np.ndarray,
        B_K: float,
        r: float,
        T: float,
        option_type: str = "call",
        basis_funcs: List[Callable[[np.ndarray], np.ndarray]] = None,
    ) -> Tuple[float, float]:
        """
        Enhanced Longstaff-Schwartz for American up-and-out barrier options
        Returns price and standard error
        """
        num_paths, num_steps_plus_one = paths.shape
        num_steps = num_steps_plus_one - 1
        dt = T / num_steps

        if basis_funcs is None:
            # Enhanced basis functions for better accuracy
            basis_funcs = [
                lambda x: np.ones_like(x),
                lambda x: x,
                lambda x: x**2,
                lambda x: x**3,
                lambda x: np.maximum(x - 1.0, 0),  # Intrinsic value
                lambda x: np.log(x) if np.all(x > 0) else np.zeros_like(x),
            ]

        # Track barrier hits
        barrier_hit = np.any(paths >= B_K, axis=1)

        # Initialize cash flows
        cash_flows = np.zeros((num_paths, num_steps_plus_one))

        # Final payoff (only if barrier not hit)
        if option_type == "call":
            final_payoff = np.maximum(paths[:, -1] - 1.0, 0)
        else:
            final_payoff = np.maximum(1.0 - paths[:, -1], 0)

        cash_flows[:, -1] = np.where(barrier_hit, 0, final_payoff)

        # Backward induction
        for t in range(num_steps - 1, 0, -1):
            # Check if barrier hit up to this point
            barrier_hit_so_far = np.any(paths[:, :t+1] >= B_K, axis=1)

            # Exercise value (zero if barrier hit)
            if option_type == "call":
                exercise_value = np.maximum(paths[:, t] - 1.0, 0)
            else:
                exercise_value = np.maximum(1.0 - paths[:, t], 0)

            exercise_value = np.where(barrier_hit_so_far, 0, exercise_value)

            # Only consider paths that are in-the-money and haven't hit barrier
            itm_and_alive = (exercise_value > 0) & (~barrier_hit_so_far)
            itm_indices = np.where(itm_and_alive)[0]

            if len(itm_indices) > 10:  # Need sufficient data points
                try:
                    X = np.column_stack([
                        f(paths[itm_indices, t]) for f in basis_funcs
                        if not (hasattr(f, '__name__') and f.__name__ == '<lambda>' and 'log' in str(f))
                        or np.all(paths[itm_indices, t] > 0)
                    ])

                    # Continuation value
                    Y = np.exp(-r * dt) * cash_flows[itm_indices, t + 1]

                    # Regularized regression for stability
                    beta, residuals, rank, s = np.linalg.lstsq(X, Y, rcond=1e-10)
                    continuation_value = np.dot(X, beta)

                    # Exercise decision
                    for i, path_idx in enumerate(itm_indices):
                        if exercise_value[path_idx] > continuation_value[i]:
                            cash_flows[path_idx, t] = exercise_value[path_idx]
                            cash_flows[path_idx, t + 1:] = 0
                        else:
                            cash_flows[path_idx, t] = 0

                except np.linalg.LinAlgError:
                    # Fallback to simple continuation
                    for path_idx in itm_indices:
                        cash_flows[path_idx, t] = 0

        # Calculate price
        discount_factors = np.exp(-r * np.arange(num_steps_plus_one) * dt)
        discounted_cash_flows = cash_flows * discount_factors[np.newaxis, :]
        option_values = np.sum(discounted_cash_flows, axis=1)

        price = np.mean(option_values)
        std_error = np.std(option_values) / np.sqrt(num_paths)

        return price, std_error

    def sample_parameter_space_optimized(
        self,
        bounds: Dict[str, Tuple[float, float]],
        num_samples: int = 1000,
        method: str = "sobol",
    ) -> Dict[str, np.ndarray]:
        """Optimized parameter space sampling with better coverage"""
        np.random.seed(self.seed)
        samples = {}

        if method == "sobol":
            try:
                from scipy.stats.qmc import Sobol
                param_names = list(bounds.keys())
                n_dims = len(param_names)

                # Ensure number of samples is power of 2 for Sobol
                sobol_samples_count = 2 ** int(np.ceil(np.log2(num_samples)))
                sampler = Sobol(d=n_dims, scramble=True, seed=self.seed)
                sobol_samples = sampler.random(n=sobol_samples_count)

                # Take only the requested number of samples
                sobol_samples = sobol_samples[:num_samples]

                for i, param in enumerate(param_names):
                    lower, upper = bounds[param]
                    samples[param] = lower + (upper - lower) * sobol_samples[:, i]

            except ImportError:
                print("Scipy not available, using Latin Hypercube")
                method = "latin_hypercube"

        if method == "latin_hypercube":
            try:
                from scipy.stats.qmc import LatinHypercube
                param_names = list(bounds.keys())
                n_dims = len(param_names)

                sampler = LatinHypercube(d=n_dims, seed=self.seed)
                lhs_samples = sampler.random(n=num_samples)

                for i, param in enumerate(param_names):
                    lower, upper = bounds[param]
                    samples[param] = lower + (upper - lower) * lhs_samples[:, i]

            except ImportError:
                print("Falling back to stratified uniform sampling")
                for param, (lower, upper) in bounds.items():
                    # Stratified sampling for better coverage
                    strata = int(np.sqrt(num_samples))
                    samples_per_stratum = num_samples // strata
                    param_samples = []

                    for i in range(strata):
                        stratum_lower = lower + i * (upper - lower) / strata
                        stratum_upper = lower + (i + 1) * (upper - lower) / strata
                        stratum_samples = np.random.uniform(
                            stratum_lower, stratum_upper, samples_per_stratum
                        )
                        param_samples.extend(stratum_samples)

                    # Fill remaining samples
                    remaining = num_samples - len(param_samples)
                    if remaining > 0:
                        param_samples.extend(
                            np.random.uniform(lower, upper, remaining)
                        )

                    samples[param] = np.array(param_samples[:num_samples])

        return samples

    def run_american_barrier_pricing(
        self,
        samples: Dict[str, np.ndarray],
        option_type: str = "call",
        use_tqdm: bool = True,
        save_results: bool = True,
        filename_prefix: str = "american_up_out",
    ) -> Dict[str, Any]:
        """
        Run American up-and-out barrier option pricing with high accuracy
        """
        num_samples = len(next(iter(samples.values())))
        prices = np.zeros(num_samples)
        errors = np.zeros(num_samples)

        start_time = time.time()
        iterator = tqdm(range(num_samples), desc="Pricing options") if use_tqdm else range(num_samples)

        for i in iterator:
            S0_K = samples["S0_K"][i]
            B_K = samples["B_K"][i]
            T = samples["T"][i]
            r = samples["r"][i]
            sigma = samples["sigma"][i]

            # Generate paths
            paths = self.generate_paths(S0_K, T, r, sigma)

            # Price American barrier option
            price, std_err = self.longstaff_schwartz_barrier(
                paths, B_K, r, T, option_type
            )

            prices[i] = price
            errors[i] = std_err

            if not use_tqdm and (i + 1) % 100 == 0:
                elapsed = time.time() - start_time
                avg_time_per_sample = elapsed / (i + 1)
                remaining_time = avg_time_per_sample * (num_samples - i - 1)
                print(f"Completed {i + 1}/{num_samples}. Est. remaining: {remaining_time:.1f}s")

        # Create results dataframe
        results_data = {**samples, "price": prices, "std_error": errors}
        results_df = pd.DataFrame(results_data)

        # Add derived metrics
        results_df['relative_error'] = results_df['std_error'] / results_df['price']
        results_df['confidence_95'] = 1.96 * results_df['std_error']
        results_df['moneyness'] = results_df['S0_K']
        results_df['barrier_distance'] = results_df['B_K'] - results_df['S0_K']

        if save_results:
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            filename = f"{filename_prefix}_{option_type}_{timestamp}.csv"
            results_df.to_csv(filename, index=False)
            print(f"Results saved to {filename}")

        return {
            "samples": samples,
            "prices": prices,
            "errors": errors,
            "option_type": option_type,
            "dataframe": results_df,
            "summary_stats": self._calculate_summary_stats(results_df),
        }

    def _calculate_summary_stats(self, df: pd.DataFrame) -> Dict:
        """Calculate summary statistics for the pricing results"""
        return {
            "price_stats": {
                "mean": df['price'].mean(),
                "std": df['price'].std(),
                "min": df['price'].min(),
                "max": df['price'].max(),
                "median": df['price'].median(),
            },
            "error_stats": {
                "mean_std_error": df['std_error'].mean(),
                "max_std_error": df['std_error'].max(),
                "mean_relative_error": df['relative_error'].mean(),
                "max_relative_error": df['relative_error'].max(),
            },
            "parameter_correlations": df[['S0_K', 'B_K', 'T', 'r', 'sigma', 'price']].corr()['price'].to_dict(),
        }

    def create_comprehensive_plots(self, results: Dict):
        """Create comprehensive visualization of results"""
        df = results["dataframe"]
        option_type = results["option_type"]

        # Create subplots
        fig = make_subplots(
            rows=2, cols=3,
            subplot_titles=[
                f"Price vs Spot/Strike ({option_type.title()})",
                f"Price vs Barrier/Strike ({option_type.title()})",
                f"Price vs Time to Maturity ({option_type.title()})",
                f"Price vs Volatility ({option_type.title()})",
                f"Price vs Interest Rate ({option_type.title()})",
                "Pricing Errors Distribution"
            ],
            specs=[[{"type": "scatter"}, {"type": "scatter"}, {"type": "scatter"}],
                   [{"type": "scatter"}, {"type": "scatter"}, {"type": "histogram"}]]
        )

        # Plot 1: Price vs S0/K
        fig.add_trace(
            go.Scatter(
                x=df['S0_K'], y=df['price'], mode='markers',
                marker=dict(color=df['T'], colorscale='Viridis', size=4),
                name='Price vs S0/K', showlegend=False
            ), row=1, col=1
        )

        # Plot 2: Price vs B/K
        fig.add_trace(
            go.Scatter(
                x=df['B_K'], y=df['price'], mode='markers',
                marker=dict(color=df['sigma'], colorscale='Plasma', size=4),
                name='Price vs B/K', showlegend=False
            ), row=1, col=2
        )

        # Plot 3: Price vs T
        fig.add_trace(
            go.Scatter(
                x=df['T'], y=df['price'], mode='markers',
                marker=dict(color=df['r'], colorscale='Cividis', size=4),
                name='Price vs T', showlegend=False
            ), row=1, col=3
        )

        # Plot 4: Price vs Volatility
        fig.add_trace(
            go.Scatter(
                x=df['sigma'], y=df['price'], mode='markers',
                marker=dict(color=df['S0_K'], colorscale='Turbo', size=4),
                name='Price vs Sigma', showlegend=False
            ), row=2, col=1
        )

        # Plot 5: Price vs Interest Rate
        fig.add_trace(
            go.Scatter(
                x=df['r'], y=df['price'], mode='markers',
                marker=dict(color=df['B_K'], colorscale='Spectral', size=4),
                name='Price vs r', showlegend=False
            ), row=2, col=2
        )

        # Plot 6: Error distribution
        fig.add_trace(
            go.Histogram(
                x=df['relative_error'], nbinsx=50,
                name='Relative Error Dist', showlegend=False
            ), row=2, col=3
        )

        fig.update_layout(
            title=f"American Up-and-Out {option_type.title()} Option Analysis",
            height=800,
            showlegend=False
        )

        fig.show()

        # Create 3D surface plot
        self._create_3d_surface_plot(df, option_type)

    def _create_3d_surface_plot(self, df: pd.DataFrame, option_type: str):
        """Create 3D surface plot for key parameters"""
        fig = go.Figure()

        fig.add_trace(go.Scatter3d(
            x=df['S0_K'],
            y=df['B_K'],
            z=df['price'],
            mode='markers',
            marker=dict(
                size=3,
                color=df['price'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Option Price")
            ),
            text=[f"T={t:.2f}, σ={s:.2f}, r={r:.3f}" for t, s, r in zip(df['T'], df['sigma'], df['r'])],
            hovertemplate="S0/K: %{x:.3f}<br>B/K: %{y:.3f}<br>Price: %{z:.4f}<br>%{text}<extra></extra>"
        ))

        fig.update_layout(
            title=f"American Up-and-Out {option_type.title()} Option Price Surface",
            scene=dict(
                xaxis_title="S0/K (Spot/Strike)",
                yaxis_title="B/K (Barrier/Strike)",
                zaxis_title="Option Price",
                camera=dict(eye=dict(x=1.2, y=1.2, z=1.2))
            ),
            width=900,
            height=700
        )

        fig.show()


def main():
    """Main execution function"""
    print("=== American Up-and-Out FX Barrier Option Pricer ===\n")

    # Initialize pricer with high precision settings
    pricer = OptimizedAmericanBarrierPricer(
        num_paths=500000,  # High path count for fourth decimal accuracy
        num_steps=252,
        antithetic=True,
        control_variate=True
    )

    # Define parameter bounds for American up-and-out options
    # These ranges are chosen to capture economically meaningful scenarios

    call_bounds = {
        "S0_K": (0.85, 1.15),    # Spot around strike (out-of-money to in-the-money)
        "B_K": (1.05, 1.50),    # Barrier above strike (typical for up-and-out)
        "T": (0.08, 2.0),       # 1 month to 2 years
        "r": (0.0, 0.08),       # 0% to 8% interest rates
        "sigma": (0.08, 0.50),  # 8% to 50% volatility (FX typical range)
    }

    put_bounds = {
        "S0_K": (0.85, 1.15),    # Same range for puts
        "B_K": (1.05, 1.50),    # Barrier above current spot
        "T": (0.08, 2.0),
        "r": (0.0, 0.08),
        "sigma": (0.08, 0.50),
    }

    # Number of samples for the hypercube
    num_samples = 5000  # Adjust based on computational resources

    print("Generating parameter samples using Sobol sequence...")

    # Generate samples for calls
    call_samples = pricer.sample_parameter_space_optimized(
        call_bounds, num_samples=num_samples, method="sobol"
    )

    # Generate samples for puts
    put_samples = pricer.sample_parameter_space_optimized(
        put_bounds, num_samples=num_samples, method="sobol"
    )

    print(f"Generated {num_samples} samples for each option type\n")

    # Price American up-and-out call options
    print("Pricing American up-and-out CALL options...")
    call_results = pricer.run_american_barrier_pricing(
        call_samples,
        option_type="call",
        use_tqdm=True,
        save_results=True,
        filename_prefix="american_up_out_call"
    )

    # Price American up-and-out put options
    print("\nPricing American up-and-out PUT options...")
    put_results = pricer.run_american_barrier_pricing(
        put_samples,
        option_type="put",
        use_tqdm=True,
        save_results=True,
        filename_prefix="american_up_out_put"
    )

    # Display results summary
    print("\n=== CALL OPTIONS SUMMARY ===")
    call_stats = call_results["summary_stats"]
    print(f"Average Price: {call_stats['price_stats']['mean']:.6f}")
    print(f"Price Range: [{call_stats['price_stats']['min']:.6f}, {call_stats['price_stats']['max']:.6f}]")
    print(f"Average Std Error: {call_stats['error_stats']['mean_std_error']:.6f}")
    print(f"Average Relative Error: {call_stats['error_stats']['mean_relative_error']:.4%}")

    print("\n=== PUT OPTIONS SUMMARY ===")
    put_stats = put_results["summary_stats"]
    print(f"Average Price: {put_stats['price_stats']['mean']:.6f}")
    print(f"Price Range: [{put_stats['price_stats']['min']:.6f}, {put_stats['price_stats']['max']:.6f}]")
    print(f"Average Std Error: {put_stats['error_stats']['mean_std_error']:.6f}")
    print(f"Average Relative Error: {put_stats['error_stats']['mean_relative_error']:.4%}")

    # Create visualizations
    print("\nGenerating visualizations...")
    pricer.create_comprehensive_plots(call_results)
    pricer.create_comprehensive_plots(put_results)

    return call_results, put_results


if __name__ == "__main__":
    call_results, put_results = main()


import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import callbacks
from keras.layers import Activation, Add, Dense, Input
from keras.models import Model
from sklearn.model_selection import train_test_split

# Fix random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)


df_list = [
    call_results["dataframe"],
    put_results["dataframe"],
]
df = pd.concat(df_list, ignore_index=True)

# Drop rows where the price is NaN and remove duplicates
df = df.dropna().drop_duplicates()

# Prepare features and target
X = df.drop("price", axis=1)  # Features
y = df["price"]  # Target variable

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)


# Define residual block function
def residual_block(x, units=100):
    shortcut = x
    out = Dense(units, activation="gelu")(x)
    out = Dense(units)(out)
    out = Add()([shortcut, out])
    out = Activation("gelu")(out)
    return out


# Build model with residual blocks
inputs = Input(shape=(X_train.shape[1],))
x = Dense(100, activation="gelu")(inputs)

for _ in range(3):
    x = residual_block(x, units=100)

outputs = Dense(1, activation="gelu")(x)
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer="adamw", loss="mae", metrics=["mae"])

# Learning rate scheduler callback
lr_scheduler = callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=5, min_lr=1e-7, verbose=1
)

# Train the model
history = model.fit(
    X_train, y_train, epochs=100, validation_split=0.2, callbacks=[lr_scheduler]
)

# Evaluate on the test set
model.evaluate(X_test, y_test)
model.save("trained_modelAM.keras")

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate absolute percentage errors
ape = np.abs((y_test - y_pred.flatten()) / y_test)

# Calculate the fraction of test samples with error <= 1%
frac_within_1pct = np.mean(ape <= 0.01)

print(f"Fraction of test samples with <= 1% absolute error: {frac_within_1pct:.4f}")

if frac_within_1pct >= 0.99:
    print("Success: At least 99% of test samples have error <= 1%.")
else:
    print("Warning: Less than 99% of test samples have error <= 1%.")